<a href="https://colab.research.google.com/github/JimKing100/DS-Unit-2-Kaggle-Challenge/blob/master/Kaggle_Challenge_Assignment_Base4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install --upgrade category_encoders plotly

In [2]:
# Imports
import os, sys

os.chdir('/content')
!git init .
!git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
!git pull origin master

!pip install -r requirements.txt

os.chdir('module1')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85
Unpacking objects: 100% (85/85), done.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 133kB 50.9MB/s 
     |████████████████████████████████| 57.7MB 34.4MB/s 
     |████████████████████████████████| 245kB 42.3MB/s 
     |████████████████████████████████| 614kB 40.1MB/s 
     |████████████████████████████████| 225kB 47.3MB/s 
     |████████████████████████████████| 768kB 45.5MB/s 
     |████████████████████████████████| 194kB 48.9MB/s 
     |████████████████████████████████| 51kB 22.7MB/s 
     |████████████████████████████████| 51kB 24.5MB/s 
     |████████████████████████████████| 737kB 43.1MB/s 
  Created wheel for pandas-profilin

In [0]:
# Disable warning
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
# Imports
import pandas as pd
import numpy as np
import math

import sklearn
sklearn.__version__

# Import the models
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline

# Import encoder and scaler and imputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Import random forest classifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
# Import, load data and split data into train, validate and test
train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

# Load initial train features and labels
from sklearn.model_selection import train_test_split
X_train = train_features
y_train = train_labels['status_group']

# Split the initial train features and labels 80% into new train and new validation
X_train, X_val, y_train, y_val = train_test_split(
  X_train, y_train, train_size = 0.80, test_size = 0.20,
  stratify = y_train, random_state=42
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [0]:
# Wrangle train, validate, and test sets
def wrangle(X):
    
    # Set bins value
    bins=20
    chars = 3
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # Create missing columns
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    
    for col in cols_with_zeros:
        print(col)
        X[col] = X[col].replace(0, np.nan)
        X[col+'_missing'] = X[col].isnull()
    
    for col in cols_with_zeros:
        X[col] = X[col].replace(np.nan, 0)
    
    # Clean installer
    X['installer'] = X['installer'].str.lower()
    X['installer'] = X['installer'].str.replace('danid', 'danida')
    X['installer'] = X['installer'].str.replace('disti', 'district council')
    X['installer'] = X['installer'].str.replace('commu', 'community')
    X['installer'] = X['installer'].str.replace('central government', 'government')
    X['installer'] = X['installer'].str.replace('kkkt _ konde and dwe', 'kkkt')
    X['installer'] = X['installer'].str[:chars]
    X['installer'].value_counts(normalize=True)
    tops = X['installer'].value_counts()[:bins].index
    X.loc[~X['installer'].isin(tops), 'installer'] = 'Other'
    
    # Clean funder and bin
    X['funder'] = X['funder'].str.lower()
    X['funder'] = X['funder'].str[:chars]
    X['funder'].value_counts(normalize=True)
    tops = X['funder'].value_counts()[:bins].index
    X.loc[~X['funder'].isin(tops), 'funder'] = 'Other'

    # Use mean for gps_height missing values
    #X.loc[X['gps_height'] == 0, 'gps_height'] = X['gps_height'].median()
    
    # Bin lga
    tops = X['lga'].value_counts()[:bins].index
    X.loc[~X['lga'].isin(tops), 'lga'] = 'Other'

    # Bin ward 
    tops = X['ward'].value_counts()[:bins].index
    X.loc[~X['ward'].isin(tops), 'ward'] = 'Other'
    
    # Bin subvillage
    tops = X['subvillage'].value_counts()[:bins].index
    X.loc[~X['subvillage'].isin(tops), 'subvillage'] = 'Other'

    # Clean latitude and longitude
    avg_lat_ward = X.groupby('ward').latitude.mean()
    avg_lat_lga = X.groupby('lga').latitude.mean()
    avg_lat_region = X.groupby('region').latitude.mean()
    avg_lat_country = X.latitude.mean()
    
    avg_long_ward = X.groupby('ward').longitude.mean()
    avg_long_lga = X.groupby('lga').longitude.mean()
    avg_long_region = X.groupby('region').longitude.mean()
    avg_long_country = X.longitude.mean()
    
    
    #cols_with_zeros = ['longitude', 'latitude']
    #for col in cols_with_zeros:
    #    X[col] = X[col].replace(0, np.nan)
    #X.loc[X['latitude'] == 0, 'latitude'] = X['latitude'].median()
    #X.loc[X['longitude'] == 0, 'longitude'] = X['longitude'].median()
    
    for i in range(0, 9): 
  
      X.loc[(X['latitude'] == 0) & (X['ward'] == avg_lat_ward.index[0]), 'latitude'] = avg_lat_ward[i]
      X.loc[(X['latitude'] == 0) & (X['lga'] == avg_lat_lga.index[0]), 'latitude'] = avg_lat_lga[i]
      X.loc[(X['latitude'] == 0) & (X['region'] == avg_lat_region.index[0]), 'latitude'] = avg_lat_region[i]
      X.loc[(X['latitude'] == 0), 'latitude'] = avg_lat_country

      X.loc[(X['longitude'] == 0) & (X['ward'] == avg_long_ward.index[0]), 'longitude'] = avg_long_ward[i]
      X.loc[(X['longitude'] == 0) & (X['lga'] == avg_long_lga.index[0]), 'longitude'] = avg_long_lga[i]
      X.loc[(X['longitude'] == 0) & (X['region'] == avg_long_region.index[0]), 'longitude'] = avg_long_region[i]
      X.loc[(X['longitude'] == 0), 'longitude'] = avg_long_country
   
    #average_lat = X.groupby('region').latitude.mean().reset_index()
    #average_long = X.groupby('region').longitude.mean().reset_index()

    #shinyanga_lat = average_lat.loc[average_lat['region'] == 'Shinyanga', 'latitude']
    #shinyanga_long = average_long.loc[average_lat['region'] == 'Shinyanga', 'longitude']

    #X.loc[(X['region'] == 'Shinyanga') & (X['latitude'] > -1), ['latitude']] = shinyanga_lat[17]
    #X.loc[(X['region'] == 'Shinyanga') & (X['longitude'] == 0), ['longitude']] = shinyanga_long[17]

    #mwanza_lat = average_lat.loc[average_lat['region'] == 'Mwanza', 'latitude']
    #mwanza_long = average_long.loc[average_lat['region'] == 'Mwanza', 'longitude']

    #X.loc[(X['region'] == 'Mwanza') & (X['latitude'] > -1), ['latitude']] = mwanza_lat[13]
    #X.loc[(X['region'] == 'Mwanza') & (X['longitude'] == 0) , ['longitude']] = mwanza_long[13]
    
    # Impute mean for tsh based on mean of source_class/basin/waterpoint_type_group
    def tsh_calc(tsh, source, base, waterpoint):
      if tsh == 0:
        if (source, base, waterpoint) in tsh_dict:
          new_tsh = tsh_dict[source, base, waterpoint]
          return new_tsh
      else:
        return tsh
      return tsh
  
    temp = X[X['amount_tsh'] != 0].groupby(['source_class',
                                            'basin',
                                            'waterpoint_type_group'])['amount_tsh'].mean()

    tsh_dict = dict(temp)
    X['amount_tsh'] = X.apply(lambda x: tsh_calc(x['amount_tsh'], x['source_class'], x['basin'], x['waterpoint_type_group']), axis=1)
    #X.loc[X['amount_tsh'] == 0, 'amount_tsh'] = X['amount_tsh'].median()
    
    # Impute mean for construction_year based on mean of source_class/basin/waterpoint_type_group
    temp = X[X['construction_year'] != 0].groupby(['source_class',
                                                   'basin',
                                                   'waterpoint_type_group'])['amount_tsh'].mean()

    tsh_dict = dict(temp)
    X['construction_year'] = X.apply(lambda x: tsh_calc(x['construction_year'], x['source_class'], x['basin'], x['waterpoint_type_group']), axis=1)
    #X.loc[X['construction_year'] == 0, 'construction_year'] = X['construction_year'].median()
    
    # Impute mean for the feature based on latitude and longitude
    def latlong_conversion(feature, pop, long, lat):
    
      radius = 0.1
      radius_increment = 0.3
    
      if pop <= 1:
        pop_temp = pop
        while pop_temp <= 1 and radius <= 2:
          lat_from = lat - radius
          lat_to = lat + radius
          long_from = long - radius
          long_to = long + radius
        
          df = X[(X['latitude'] >= lat_from) & 
                 (X['latitude'] <= lat_to) &
                 (X['longitude'] >= long_from) &
                 (X['longitude'] <= long_to)]
        
          pop_temp = df[feature].mean()
          if math.isnan(pop_temp):
            pop_temp = pop
          radius = radius + radius_increment
      else:
        pop_temp = pop
      
      if pop_temp <= 1:
        new_pop = X_train[feature].mean()
      else:
        new_pop = pop_temp
        
      return new_pop
    
    # Impute population based on location
    #X['population'] = X.apply(lambda x: latlong_conversion('population', x['population'], x['longitude'], x['latitude']), axis=1)
    #X.loc[X['population'] == 0, 'population'] = X['population'].median()
    
    # Impute gps_height based on location
    X['gps_height'] = X.apply(lambda x: latlong_conversion('gps_height', x['gps_height'], x['longitude'], x['latitude']), axis=1)
    
    # Drop recorded_by (never varies) and id (always varies, random) and num_private (empty)
    unusable_variance = ['recorded_by', 'id', 'num_private','wpt_name', 'extraction_type_class',
                         'quality_group', 'source_type', 'source_class', 'waterpoint_type_group']
    X = X.drop(columns=unusable_variance)
    
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type', 'extraction_type_group']
    X = X.drop(columns=duplicates)
    
    # return the wrangled dataframe
    return X


In [7]:
# Wrangle the data
X_train = wrangle(X_train)
X_val = wrangle(X_val)

longitude
latitude
construction_year
gps_height
population
longitude
latitude
construction_year
gps_height
population


In [0]:
# Feature engineering
def feature_engineer(X):
  
  # Create new feature pump_age
  X['pump_age'] = 2013 - X['construction_year']
  X.loc[X['pump_age'] == 2013, 'pump_age'] = 0
  X.loc[X['pump_age'] == 0, 'pump_age'] = 10
  
  
  # Convert date_recorded to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
  # Extract components from date_recorded, then drop the original column
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
    
  # Engineer feature: how many years from construction_year to date_recorded
  X['years'] = X['year_recorded'] - X['construction_year']
  X['years_missing'] = X['years'].isnull()
  
  column_list = ['date_recorded']
  X = X.drop(columns=column_list)
  
  # Create new feature region_district
  #X['region_district'] = X['region_code'].astype(str) + X['district_code'].astype(str)
  
  #X['tsh_pop'] = X['amount_tsh']/X['population']

  return X

In [0]:
# Feature engineer the data
X_train = feature_engineer(X_train)
X_val = feature_engineer(X_val)

In [10]:
X_train.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,management,management_group,payment,water_quality,quantity,source,waterpoint_type,longitude_missing,latitude_missing,construction_year_missing,gps_height_missing,population_missing,pump_age,year_recorded,month_recorded,day_recorded,years,years_missing
43360,1415.482234,Other,20.655640,Other,33.542898,-9.174777,Lake Nyasa,Other,Mbeya,12,4,Rungwe,Other,0.0,True,VWC,K,NaN,1415.110875,gravity,vwc,user-group,never pay,soft,insufficient,spring,communal standpipe,False,False,True,True,True,597.889125,2011,7,27,595.889125,False
7263,500.000000,Other,2049.000000,Other,34.665760,-9.308548,Rufiji,Other,Iringa,11,4,Njombe,Imalinyi,175.0,True,WUA,Tove Mtwango gravity Scheme,True,2008.000000,gravity,wua,user-group,pay monthly,soft,enough,spring,communal standpipe,False,False,False,False,False,5.000000,2011,3,23,3.000000,False
2486,25.000000,Other,290.000000,Other,38.238568,-6.179919,Wami / Ruvu,Other,Pwani,6,1,Bagamoyo,Other,2300.0,True,VWC,NaN,False,2010.000000,india mark ii,vwc,user-group,pay per bucket,salty,insufficient,shallow well,hand pump,False,False,False,False,False,3.000000,2011,3,7,1.000000,False
313,391.538462,gov,4.781262,dwe,30.716727,-1.289055,Lake Victoria,Other,Kagera,18,1,Karagwe,Other,0.0,True,NaN,NaN,True,389.075055,other,vwc,user-group,never pay,soft,enough,shallow well,other,False,False,True,True,True,1623.924945,2011,7,31,1621.924945,False
52726,1267.552189,wat,67.488372,gov,35.389331,-6.399942,Internal,Other,Dodoma,1,6,Other,Other,0.0,True,VWC,Zeje,True,1270.526091,mono,vwc,user-group,pay per bucket,soft,enough,machine dbh,communal standpipe,False,False,True,True,True,742.473909,2011,3,10,740.473909,False


In [0]:
# Encode a feature
def encode_feature(X, y, str):
  X['status_group'] = y
  X.groupby(str)['status_group'].value_counts(normalize=True)
  X['functional']= (X['status_group'] == 'functional').astype(int)
  X[['status_group', 'functional']]
  return X

In [0]:
# Encode all the categorical features
train = X_train.copy()
train = encode_feature(train, y_train, 'quantity')
train = encode_feature(train, y_train, 'waterpoint_type')
train = encode_feature(train, y_train, 'extraction_type')
train = encode_feature(train, y_train, 'installer')
train = encode_feature(train, y_train, 'funder')
train = encode_feature(train, y_train, 'water_quality')
train = encode_feature(train, y_train, 'basin')
train = encode_feature(train, y_train, 'region')
train = encode_feature(train, y_train, 'payment')
train = encode_feature(train, y_train, 'source')
train = encode_feature(train, y_train, 'lga')
train = encode_feature(train, y_train, 'ward')
train = encode_feature(train, y_train, 'scheme_management')
train = encode_feature(train, y_train, 'management')
#train = encode_feature(train, y_train, 'region_district')
train = encode_feature(train, y_train, 'subvillage')

In [13]:
# use quantity feature and the numerical features but drop id
categorical_features = ['quantity', 'waterpoint_type', 'extraction_type', 'installer',
                        'funder', 'water_quality', 'basin', 'region', 'payment', 
                        'source', 'lga', 'ward', 'scheme_management', 'management', 'subvillage',
                        'permit']
                                              
#numeric_features = X_train.select_dtypes('number').columns.drop('id').tolist()
numeric_features = X_train.select_dtypes('number').columns.tolist()
features = categorical_features + numeric_features

# make subsets using the quantity feature all numeric features except id
X_train = X_train[features]
X_val = X_val[features]

# Create the logistic regression pipeline
pipeline = make_pipeline (
    ce.OneHotEncoder(use_cat_names=True),
    #SimpleImputer(),
    StandardScaler(),
    LogisticRegressionCV(random_state=42, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

print('Validation Accuracy', pipeline.score(X_val, y_val))         

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Validation Accuracy 0.7407407407407407


In [14]:
features

['quantity',
 'waterpoint_type',
 'extraction_type',
 'installer',
 'funder',
 'water_quality',
 'basin',
 'region',
 'payment',
 'source',
 'lga',
 'ward',
 'scheme_management',
 'management',
 'subvillage',
 'permit',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'region_code',
 'district_code',
 'population',
 'construction_year',
 'pump_age',
 'year_recorded',
 'month_recorded',
 'day_recorded',
 'years']

In [15]:
# Create the random forest pipeline
pipeline = make_pipeline (
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    StandardScaler(),
    RandomForestClassifier(n_estimators=1400, 
                           random_state=42,
                           min_samples_split=5,
                           min_samples_leaf=1,
                           max_features='auto',
                           max_depth=30,
                           bootstrap=True,
                           n_jobs=-1,
                           verbose = 1)
)

pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))         

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed:  1.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    2.4s


Validation Accuracy 0.8136363636363636


[Parallel(n_jobs=4)]: Done 1400 out of 1400 | elapsed:    2.7s finished


In [16]:
pd.set_option('display.max_rows', 200)
model = pipeline.named_steps['randomforestclassifier']
encoder = pipeline.named_steps['onehotencoder']
encoded_columns = encoder.transform(X_train).columns 
importances = pd.Series(model.feature_importances_, encoded_columns)
importances.sort_values(ascending=False)

quantity_dry                                   7.884703e-02
longitude                                      7.253572e-02
latitude                                       7.125452e-02
gps_height                                     6.467052e-02
day_recorded                                   3.937246e-02
years                                          3.310874e-02
pump_age                                       3.269586e-02
construction_year                              3.234113e-02
waterpoint_type_other                          3.077931e-02
population                                     3.063416e-02
extraction_type_other                          2.990602e-02
quantity_enough                                2.892091e-02
amount_tsh                                     2.406110e-02
district_code                                  1.674614e-02
month_recorded                                 1.620167e-02
payment_never pay                              1.498292e-02
region_code                             

In [17]:
test_features['pump_age'] = 2013 - test_features['construction_year']
test_features.loc[test_features['pump_age'] == 2013, 'pump_age'] = 0
test_features.loc[test_features['pump_age'] == 0, 'pump_age'] = 10
  
test_features['region_district'] = test_features['region_code'].astype(str) + test_features['district_code'].astype(str)

test_features['tsh_pop'] = test_features['amount_tsh']/test_features['population']

test_features.drop(columns=['num_private'])

X_test = test_features[features]

assert all(X_test.columns == X_train.columns)

y_pred = pipeline.predict(X_test)

KeyError: ignored

In [0]:
#submission = sample_submission.copy()
#submission['status_group'] = y_pred
#submission.to_csv('/content/submission-01.csv', index=False)